In [1]:
import numpy as np
import pandas as pd
import unicodedata
import re

from bs4 import BeautifulSoup
from markdown import markdown

In [2]:
ruta_data = "../data"
ruta_train = ruta_data + "/train.csv"
ruta_test = ruta_data + "/test.csv"

In [3]:
def markdown_to_text(s):
    return s.replace("*", " ")

In [4]:
"""def markdown_to_text(markdown_string):
    ''' Converts a markdown string to plaintext '''

    # md -> html -> text since BeautifulSoup can extract text cleanly
    html = markdown(markdown_string)

    # remove code snippets
    html = re.sub(r'<pre>(.*?)</pre>', ' ', html)
    html = re.sub(r'<code>(.*?)</code >', ' ', html)

    # extract text
    soup = BeautifulSoup(html, "html.parser")
    text = ''.join(soup.findAll(text=True))

    return text"""

'def markdown_to_text(markdown_string):\n    \'\'\' Converts a markdown string to plaintext \'\'\'\n\n    # md -> html -> text since BeautifulSoup can extract text cleanly\n    html = markdown(markdown_string)\n\n    # remove code snippets\n    html = re.sub(r\'<pre>(.*?)</pre>\', \' \', html)\n    html = re.sub(r\'<code>(.*?)</code >\', \' \', html)\n\n    # extract text\n    soup = BeautifulSoup(html, "html.parser")\n    text = \'\'.join(soup.findAll(text=True))\n\n    return text'

In [4]:
def strip_accents(text):

    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass

    text = unicodedata.normalize('NFD', text)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")

    return str(text)

In [5]:
train_origi = pd.read_csv(ruta_train, usecols=["titulo"])
test_origi = pd.read_csv(ruta_test, usecols=["titulo"])

In [6]:
train_origi = train_origi.fillna("")
test_origi = test_origi.fillna("")

### Veamos que forma tienen las descripciones actuales:

In [7]:
train_origi.loc[349].titulo

''

In [8]:
train_origi.loc[302].titulo

'casa en venta tlalnepantla san juan ixtacala.'

In [9]:
train_origi.loc[802].titulo

'venta de casa en queretaro**el mirador**'

In [10]:
train_origi["titulo"] = train_origi.apply(lambda x: BeautifulSoup(x["titulo"], "lxml").text, axis=1)
test_origi["titulo"] = test_origi.apply(lambda x: BeautifulSoup(x["titulo"], "lxml").text, axis=1)

/home/julian/.local/lib/python3.7/site-packages/bs4/__init__.py:272: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/home/julian/.local/lib/python3.7/site-packages/bs4/__init__.py:335: UserWarning: "http://mashogaresinmobiliaria.inmo.co/" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


### Veamos como quedaron las descripciones ahora:

In [11]:
train_origi.loc[349].titulo

''

In [12]:
train_origi.loc[302].titulo

'casa en venta tlalnepantla san juan ixtacala.'

In [13]:
train_origi.loc[802].titulo

'venta de casa en queretaro**el mirador**'

### Quitamos caracteres de fin de linea, non breaking space(\xa0), etc

In [14]:
train_origi["titulo"] = train_origi.apply(lambda x: u' '.join(x["titulo"].split()), axis=1)
test_origi["titulo"] = test_origi.apply(lambda x: u' '.join(x["titulo"].split()), axis=1)

### Parece que hay algunos tags estilo markdown, veamos de quitar esos

In [15]:
train_origi["titulo"] = train_origi["titulo"].apply(lambda x: markdown_to_text(x))
test_origi["titulo"] = test_origi["titulo"].apply(lambda x: markdown_to_text(x))

### Quitamos acentos que hayan quedado

In [21]:
#train_origi["descripcion"] = train_origi["descripcion"].apply(lambda x: strip_accents(x))
#test_origi["descripcion"] = test_origi["descripcion"].apply(lambda x: strip_accents(x))

In [16]:
train_origi.to_csv(f"{ruta_data}/titulos_train_limpios.csv", index=False)

In [17]:
test_origi.to_csv(f"{ruta_data}/titulos_test_limpios.csv", index=False)

In [18]:
train_origi[train_origi["titulo"]==""].shape

(5393, 1)

In [19]:
prue = pd.read_csv(f"{ruta_data}/titulos_train_limpios.csv")

In [20]:
prue.isna().sum()

titulo    5393
dtype: int64